In [102]:
from tqdm import tqdm
import numpy as np
import re
from queue import SimpleQueue
from functools import reduce
from shapely import Polygon, LineString
import math

with open("aoc.txt") as file:
    data = file.read()

In [106]:
def fix_workflow(w):

    rules = w[1:-1].split(",")
    end = rules[-1]
    rules = [a.split(":") for a in rules[:-1]]
    temp = {'rules' : rules, "end" : end}

    return temp

workflows, ratings = data.split('\n\n')
ratings = list(map(eval, re.sub(r"(?=[a-z])|(?<=[a-z])",'"', ratings.replace("=", ":")).splitlines()))
workflows = {(split := re.split("(?={)", a))[0] : fix_workflow(split[1]) for a in workflows.splitlines()}

In [108]:
def check_rule(rating, rule):
    letter = rule[0]
    comp = rule[1]
    nr = int(rule[2:])

    if (comp == ">" and rating[letter] > nr) or (comp == "<" and rating[letter] < nr):
        return True
    
    return False

def get_next(key, rating):

    if key in ['A','R']:
        return key

    workflow = workflows[key]

    for rule in workflow['rules']:
        if check_rule(rating, rule[0]):
            return get_next(rule[1], rating)

    return get_next(workflow['end'], rating)

sum([sum(rating.values()) for rating in ratings if get_next("in", rating) == "A"])

397134